### Train BERT from Scratch using Transformers in Python
* https://www.thepythoncode.com/article/pretraining-bert-huggingface-transformers-in-python

In [ ]:
# !pip install datasets transformers==4.18.0 sentencepiece

In [77]:
import os
import json
import re
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import *
from tokenizers import *
from datasets import *
from sklearn.model_selection import train_test_split
import nltk
from nltk.data import load

### Picking a Dataset
* sentence tokenize

In [21]:
sent_tokenizer = load("tokenizers/punkt/english.pickle")
extra_abbreviations = [
    'RE','re','pat', 'no', 'nos','vol','jan','feb','mar','apr','jun',
    'jul','aug','sep','oct','nov','dec','eng','ser','ind','ed','pp',
    'e.g','al','T.E.N.S', 'E.M.S','F.E','U.H.T.S.T','degree',
    '/gm','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
    'P','Q','R','S','T','U','V','W','X','Y','Z']
sent_tokenizer._params.abbrev_types.update(extra_abbreviations)

In [ ]:
# 김명선 책임의 문장 분리 코드
REG_SENT_KO=r'([ㄱ-ㅣ가-힣]+[.]|[\n]|[:;!?])'
REG_SENT_EN=r'([a-zA-Z]+[.]\s|[\n]|[:;!?])'

def split_sentence(doc, regex):
    s = 0
    for m in re.finditer(regex, doc):
        sent = doc[s:m.end()].strip()
        s = m.end()
        if not sent:
            continue
        yield sent

    if s < len(doc):
        sent = doc[s:].strip()
        if sent:
            yield sent

In [86]:
?unicodedata.normalize

In [116]:
data = []
kiwee_files = ['data/c09k_0001-1000.xlsx', 'data/c09k_1001-2000.xlsx', 'data/c09k_2001-3000.xlsx', 'data/c09k_3001-3935.xlsx']
with open('data/c09k_corpus.txt', 'a') as f:
    f.truncate(0)
    for i, fn in enumerate(kiwee_files):
        tmp = pd.read_excel(fn).fillna('')
        # pandas는 비어있는 컬럼의 dtype을 float로 바꿔서 인식한다. 그로 인해 토크나이징 할 데이터가 없으면 오류가 발생되어 fillna를 사용해 모두 텍스트로 인식시키도록 한다
        # https://stackoverflow.com/questions/53953286/pandas-read-excel-blanks-in-string-columns-convert-to-floats-converting-via-st
        col_text = ['발행번호', '발명의명칭', '요약', '대표청구항', '과제', '해결방안']
        tmp = tmp[col_text]
        for index, row in tmp.iterrows():
    #         print(index, '\n', row['발명의명칭'], row['요약'], row['대표청구항'], row['과제'], row['해결방안'], '\n')
            for col in col_text[1:]:
    #             print('처리중인 데이터:', col, row[col], '\n')
                if row[col].strip() == "":
                    pass
                else:
    #                 print(row[col].strip())
    #                     row[col] = unicodedata.normalize('NFKC', row[col])
                    row[col] = unicodedata.normalize('NFC', row[col])  # 자음과 모음이 깨질 때는 NFC로 변환
    #                     row[col] = unicodedata.normalize('NFKD', row[col])
                    #     https://blog.naver.com/PostView.nhn?blogId=duswl0319&logNo=221516880642&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
                    row[col] = row[col].replace('\n\t',' ')
                    row[col] = row[col].replace('\n',' ')
    #                 row[col] = row[col].replace('&lt;',' ')
                    row[col] = row[col].replace('\t\t',' ')
                    row[col] = row[col].replace('.  .','.')
                    row[col] = row[col].replace('. .','.')
                    row[col] = row[col].replace('..','.')
                    row[col] = row[col].replace(' . ','.')
                    row[col] = row[col].replace(' ． ','.')
                    row[col] = row[col].replace('． ','.')
                    row[col] = row[col].replace('. ','.')
                    row[col] = row[col].replace('  ',' ')
                    row[col] = row[col].replace('  ',' ')
                    row[col] = row[col].replace('【과제】',' ')
                    row[col] = row[col].replace('【해결 수단】',' ')
                    str_tmp = sent_tokenizer.tokenize(row[col].strip())
    #                 print('문장 분리: ', str_tmp, '\n'*3)
    #                 result  = [f"{line}\n" for line in str_tmp]
                    for line in str_tmp:
                        f.write(f"{line}\n")
#                 data.extend(result)
#                 print('누적된 문장수: ', len(data), '\n')
# data1 = np.array(data)
# np.savetxt('data/c09k_corpus.txt', data1, fmt='<U1176 %.18e', header='', footer='')

In [117]:
dataset = Dataset.from_text('data/c09k_corpus.txt')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1317.72it/s]
                            

Dataset text downloaded and prepared to /home/hdh/.cache/huggingface/datasets/text/default-f402c51ede181d32/0.0.0. Subsequent calls will reuse this data.


In [118]:
print(dataset['text'][0:5])
print(dataset['text'][-5:])

['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 &lt; x &lt; 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1@@발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2@@발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
['전기 화학 디바이스 및 그것을 형성하는 방법', '전기 화학 디바이스에 대해서, 개시한다.전기 화학 디바이스는, 제 1의 투명 도전층, 제 1의 투명 도전층 위에 있는 에렉트로 크로믹층, 에렉트로 크로믹층 위에 있는 대향 전극층, 제 2의 투명 도전층을 갖추어 23℃으로 0.68초/mm이하의 스위칭 속도 파라미터를 가진다.', '제 1의 투명 도전층과 전기 제 1의 투명 도전층 위에 있는 음극 전기 화학층과 전기

In [85]:
"""
['사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며, 산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.',
 '６５〜２． 05의 범위내인 원소 A와 폴링의 전기 음성도가 2.',
 '１０〜２． 54의 범위내인 원소 B만이 함유 되어 바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']
 위와 같이 소수점 앞 뒤로 공백이 존재하는데, 이로 인해 소수점에서 문장 분리가 되버린다. 오류 조치가 필요
 """
# 아래와 같이 구두점 앞뒤 블랭크를 제거하여 일단 구분
# row[col] = unicodedata.normalize('NFKC', row[col])
# https://blog.naver.com/PostView.nhn?blogId=duswl0319&logNo=221516880642&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
# row[col] = row[col].replace(' . ','.')
# row[col] = row[col].replace(' ． ','.')
# row[col] = row[col].replace('． ','.')
# row[col] = row[col].replace('. ','.')
# row[col] = row[col].replace('  ',' ')

"\n['사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며, 산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.',\n '６５〜２． 05의 범위내인 원소 A와 폴링의 전기 음성도가 2.',\n '１０〜２． 54의 범위내인 원소 B만이 함유 되어 바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']\n 위와 같이 소수점 앞 뒤로 공백이 존재하는데, 이로 인해 소수점에서 문장 분리가 되버린다. 오류 조치가 필요\n "

In [84]:
data

[]

In [2]:
# 과거 데이터 처리
data = pd.read_excel('data/c09k_biblio(861건).xlsx')
data = data[['발행번호', '발명의명칭', '요약', '대표청구항', '과제', '해결방안']]
# data_train, data_test = train_test_split(data, test_size=0.1, random_state=15, shuffle=True)

In [6]:
# print(data_train.shape, data_test.shape)

In [3]:
def data_pre_process(data):
    data['text'] = data[['발명의명칭', '요약', '대표청구항', '과제', '해결방안']].apply(". ".join, axis=1)
    data['text'] = data['text'].str.replace('\n\t',' ')
    data['text'] = data['text'].str.replace('\t\t',' ')
    data['text'] = data['text'].str.replace('.  .','.')
    data['text'] = data['text'].str.replace('. .','.')
    data['text'] = data['text'].str.replace('..','.')
    data['text'] = data['text'].str.replace('  ',' ')
    data['text'] = data['text'].str.strip()
    data1 = data[['발행번호', 'text']].copy()
#     data1.set_index('발행번호', inplace=True)
    return data1

In [ ]:

load_file=open('./input.txt','r')
save_file=open('./output.txt','w')
no_blank = False
while True:
    line = load_file.readline()
    if line == "":
        break
    if line.strip() == "":
        if no_blank:
            continue
        save_file.write(f"{line}")
    else:
        print(line)
        result_ = tokenizer.tokenize(line)
        print(result_)
        result  = [ f"{cur_line}\n" for cur_line in result_ ]
        for save_line in result:
            save_file.write(save_line)

In [5]:
data_text = data_pre_process(data)


/tmp/ipykernel_1638/1848036980.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('.  .','.')
/tmp/ipykernel_1638/1848036980.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('. .','.')
/tmp/ipykernel_1638/1848036980.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('..','.')


In [4]:
dataset = Dataset.from_pandas(data_text)

/tmp/ipykernel_1638/1848036980.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('.  .','.')
/tmp/ipykernel_1638/1848036980.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('. .','.')
/tmp/ipykernel_1638/1848036980.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('..','.')


In [ ]:
Dataset.fr

In [9]:
d = dataset.train_test_split(test_size=0.1)
d["train"], d["test"]
for t in d["train"]["text"][:3]:
    print(t)
    print("="*50)

복합 재료, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치. 유기 화합물과 무기 화합물을 복합한 복합 재료로서, 캐리어 수소성이 높은 복합 재료를 제공한다. 또한, 유기 화합물로의 캐리어 주입성이 높은 복합 재료를 제공한다. 또한, 전하 이동 상호 작용에 의한 흡수가 발생하기 어려운 복합 재료를 제공한다. 또한, 가시광에 대한 투광성이 높은 복합 재료를 제공한다. 
나프탈렌 골격, 페난트렌 골격, 또는 트리페닐렌 골격에 치환기가 결합한, 분자량이 350 이상 2000 이하인 탄화수소 화합물과 당해 탄화수소 화합물에 대해 전자 수용성을 나타내는 무기 화합물을 함유하고, 당해 치환기가 갖는 환은, 벤젠 환, 나프탈렌 환, 페난트렌 환, 트리페닐렌 환으로부터 선택되는 1종 또는 복수종인 복합 재료를 제공한다.  . 비페닐기를 갖는 탄화수소 화합물; 및억셉터성 물질 (acceptor substance)을 포함하는, 복합 재료로서,상기 비페닐기는 나프탈렌 골격, 페난트렌 골격, 및 트리페닐렌 골격으로부터 선택되는 복수의 치환기에 결합되고,상기 탄화수소 화합물의 분자량은 350 이상 2000 이하인, 복합 재료.. 1. 발명의 일 형태의 복합 재료인 경우에는, 결정화를 억제할 목적으로 무기 화합물의 비율을 높게 할 필요가 없으며, 전하 이동 상호 작용에 유래하는 흡수 피크가 가시광 영역에 관측되는 것을 방지할 수 있다.. 1. 비페닐기를 갖는 탄화수소 화합물; 및억셉터성 물질 (acceptor substance)을 포함하는, 복합 재료로서,상기 비페닐기는 나프탈렌 골격, 페난트렌 골격, 및 트리페닐렌 골격으로부터 선택되는 복수의 치환기에 결합되고,상기 탄화수소 화합물의 분자량은 350 이상 2000 이하인, 복합 재료.
다환 화합물 및 이를 포함하는 유기 발광 소자. 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다. . 하기 화학식 1로 표시되는 화합물:[화학식 1] 상기 화학식 1에 있어서,Ar1 내지 Ar4는 서로 같거나

In [ ]:
# data_train1 = data_pre_process(data_train)
# data_test1 = data_pre_process(data_test)

In [ ]:
# # download and prepare cc_news dataset
# dataset = load_dataset("cc_news", split="train")
# dataset[0]
# # split the dataset into training (90%) and testing (10%)
# d = dataset.train_test_split(test_size=0.1)
# d["train"], d["test"]
# for t in d["train"]["text"][:3]:
#     print(t)
#     print("="*50)

### using personal dataset
* https://huggingface.co/docs/datasets/dataset_script

In [ ]:
# # if you have huge custom dataset separated into files
# # load the splitted files
# files = ["train1.txt", "train2.txt"] # train3.txt, etc.
# dataset = load_dataset("text", data_files=files, split="train")

### Training the Tokenizer

In [10]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
    """Utility function to save dataset text to disk,
    useful for using the texts to train the tokenizer 
    (as the tokenizer accepts files)"""
    with open(output_filename, "w") as f:
        for t in dataset["text"]:
            print(t, file=f)

In [11]:
# dataset_to_text(ds, "c09k_pre_train.txt")

In [12]:
dataset_to_text(d["train"], "c09k_pre_train.txt")
dataset_to_text(d["test"], "c09k_pre_test.txt")

In [13]:
# # save the training set to train.txt
# dataset_to_text(d["train"], "train.txt")
# # save the testing set to test.txt
# dataset_to_text(d["test"], "test.txt")

In [14]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["c09k_pre_train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 15072
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = False

In [15]:
# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [16]:
model_path = "c09k_pretrained_bert"

In [18]:
# make the directory if not already there
if not os.path.isdir(model_path):
    os.mkdir(model_path)
# save the tokenizer  
tokenizer.save_model(model_path)
tokenizer.save('c09k_pretrained_bert/tokenizer.json')

In [19]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
    tokenizer_cfg = {"do_lower_case": True,
                     "unk_token": "[UNK]",
                     "sep_token": "[SEP]",
                     "pad_token": "[PAD]",
                     "cls_token": "[CLS]",
                     "mask_token": "[MASK]",
                     "model_max_length": max_length,
                     "max_len": max_length,
                    }
    json.dump(tokenizer_cfg, f)

In [20]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file c09k_pretrained_bert/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 512,
  "max_position_embeddings": 512,
  "model_max_length": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PAD]",
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sep_token": "[SEP]",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "unk_token": "[UNK]",
  "use_cache": true,
  "vocab_size": 30522
}



vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


### Tokenizing the Dataset

In [21]:
def encode_with_truncation(examples):
    """Mapping function to tokenize the sentences passed with truncation"""
    return tokenizer(examples["text"], truncation=True, padding="max_length",
                                     max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
    """Mapping function to tokenize the sentences passed without truncation"""
    return tokenizer(examples["text"], return_special_tokens_mask=True)

In [22]:
# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

In [23]:
# # tokenizing the train dataset
# train_dataset = data_train1['text'].map(encode, batched=True)
# test_dataset = data_test1['text'].map(encode, batched=True)

In [24]:
# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

100%|██████████| 1/1 [00:00<00:00, 24.07ba/s]


In [25]:
print(d["train"][:2], '\n', train_dataset[:2])

{'발행번호': ['KR10002269040B1', 'KR10002318136B1'], 'text': ['복합 재료, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치. 유기 화합물과 무기 화합물을 복합한 복합 재료로서, 캐리어 수소성이 높은 복합 재료를 제공한다. 또한, 유기 화합물로의 캐리어 주입성이 높은 복합 재료를 제공한다. 또한, 전하 이동 상호 작용에 의한 흡수가 발생하기 어려운 복합 재료를 제공한다. 또한, 가시광에 대한 투광성이 높은 복합 재료를 제공한다. \n나프탈렌 골격, 페난트렌 골격, 또는 트리페닐렌 골격에 치환기가 결합한, 분자량이 350 이상 2000 이하인 탄화수소 화합물과 당해 탄화수소 화합물에 대해 전자 수용성을 나타내는 무기 화합물을 함유하고, 당해 치환기가 갖는 환은, 벤젠 환, 나프탈렌 환, 페난트렌 환, 트리페닐렌 환으로부터 선택되는 1종 또는 복수종인 복합 재료를 제공한다.  . 비페닐기를 갖는 탄화수소 화합물; 및억셉터성 물질 (acceptor substance)을 포함하는, 복합 재료로서,상기 비페닐기는 나프탈렌 골격, 페난트렌 골격, 및 트리페닐렌 골격으로부터 선택되는 복수의 치환기에 결합되고,상기 탄화수소 화합물의 분자량은 350 이상 2000 이하인, 복합 재료.. 1. 발명의 일 형태의 복합 재료인 경우에는, 결정화를 억제할 목적으로 무기 화합물의 비율을 높게 할 필요가 없으며, 전하 이동 상호 작용에 유래하는 흡수 피크가 가시광 영역에 관측되는 것을 방지할 수 있다.. 1. 비페닐기를 갖는 탄화수소 화합물; 및억셉터성 물질 (acceptor substance)을 포함하는, 복합 재료로서,상기 비페닐기는 나프탈렌 골격, 페난트렌 골격, 및 트리페닐렌 골격으로부터 선택되는 복수의 치환기에 결합되고,상기 탄화수소 화합물의 분자량은 350 이상 2000 이하인, 복합 재료.', '다환 화합물 및 이를 포함하는 유기 발광 소자. 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다

In [26]:
if truncate_longer_samples:
    # remove other columns and set input_ids and attention_mask as PyTorch tensors
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
    # remove other columns, and remain them as Python lists
    test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
    train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

In [27]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

In [28]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

In [29]:
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
# https://pytorch.org/get-started/locally/#windows-package-manager

In [30]:
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
    train_dataset = train_dataset.map(group_texts, batched=True,
                                                                        desc=f"Grouping texts in chunks of {max_length}")
    test_dataset = test_dataset.map(group_texts, batched=True,
                                                                    desc=f"Grouping texts in chunks of {max_length}")
    # convert them from lists to torch tensors
    train_dataset.set_format("torch")
    test_dataset.set_format("torch")

Grouping texts in chunks of 512: 100%|██████████| 1/1 [00:00<00:00, 16.24ba/s]


In [31]:
len(train_dataset), len(test_dataset)

(1049, 135)

### Loading the Model

In [32]:
vocab_size

15072

In [33]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

### Training

In [34]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [35]:
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=8,  # evaluation batch size
    logging_steps=10,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=10,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 10
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
model_path

'c09k_pretrained_bert'

In [37]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [38]:
# train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1049
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 130


Step,Training Loss,Validation Loss
10,8.463800,7.923255
20,7.833800,7.537686
30,7.440900,7.227184
40,7.128600,6.954088
50,6.805100,6.723242
60,6.616200,6.485281
70,6.443300,6.311793
80,6.321800,6.222284
90,6.129300,6.170388
100,6.135000,6.089207


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 135
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert/checkpoint-10
Configuration saved in c09k_pretrained_bert/checkpoint-10/config.json
Model weights saved in c09k_pretrained_bert/checkpoint-10/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 135
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert/checkpoint-20
Configuration saved in c09k_pretrained_bert/checkpoint-20/config.json
Mo

TrainOutput(global_step=130, training_loss=6.722118172278771, metrics={'train_runtime': 702.5491, 'train_samples_per_second': 14.931, 'train_steps_per_second': 0.185, 'total_flos': 2758152189542400.0, 'train_loss': 6.722118172278771, 'epoch': 9.99})

### Using the Model

In [39]:
# load the model checkpoint
model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-130"))
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_path)

loading configuration file c09k_pretrained_bert/checkpoint-130/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 15072
}

loading weights file c09k_pretrained_bert/checkpoint-130/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c09k_pretrained_bert/checkpoint-130.
If your task is similar to the task th

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [40]:
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [42]:
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
]
for example in examples:
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

인광성 유기 금속 이리듐 착체,, 소자, 발광 장치, 전자 기기, 및 조명 장치, confidence: 0.29707568883895874
인광성 유기 금속 이리듐 착체, - 소자, 발광 장치, 전자 기기, 및 조명 장치, confidence: 0.016033636406064034
인광성 유기 금속 이리듐 착체,. 소자, 발광 장치, 전자 기기, 및 조명 장치, confidence: 0.015503378584980965
인광성 유기 금속 이리듐 착체, 또는 소자, 발광 장치, 전자 기기, 및 조명 장치, confidence: 0.01475199218839407
인광성 유기 금속 이리듐 착체, 및 소자, 발광 장치, 전자 기기, 및 조명 장치, confidence: 0.012600366026163101
본 명세서는 화학식 1로 표시되는. 및 이를 포함하는 유기 발광 소자에 관한 것이다, confidence: 0.03568422794342041
본 명세서는 화학식 1로 표시되는, 및 이를 포함하는 유기 발광 소자에 관한 것이다, confidence: 0.03509819507598877
본 명세서는 화학식 1로 표시되는 ; 및 이를 포함하는 유기 발광 소자에 관한 것이다, confidence: 0.031512174755334854
본 명세ᄉ